# ChiSquare test/distribution for general fits:

This Python program/notebook illustrates the use of ChiSquare as a goodness-of-fit measure, how this distribution comes about, and that it actually works, here with three different examples! The first example is the linear fit, while the two others are more complicated (oscillatory graph fit and exponential fit of a histogram). However, they have one thing in common, namely the number of degrees of freedom!

## References:
* Barlow: Chapter 6
* Cowan: Chapter 2.7, Chapter 7
* Bevington: Chapter 6

## Author(s), contact(s), and dates:
* Author: Troels C. Petersen (NBI)
* Email:  petersen@nbi.dk
* Date:   10th of November 2025

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from iminuit import Minuit
from iminuit.cost import LeastSquares

***
Make sure you've read the relevant references and that you understand not only what
the ChiSquare is, but also that it follows the ChiSquare distribution, and that the
probability of obtaining such a ChiSquare or worse can be calculated from it.

The program generates a certain number of datasets in three different ways, from
which the Chi2 of the fit is recorded.
***

## Program settings:

In [ ]:
Minuit.print_level = 0      # Don't print the fit result (many fits to do!)
save_plots = False

r = np.random                         # Random generator
r.seed(42)

## Generate and fit LINEAR data:

In [ ]:
Nexp = 1000
NpointsLin = 17

In [ ]:
alpha0 = 3.6
alpha1 = 0.3
sigmay = 0.4

In [ ]:
array_Chi2_Lin = np.zeros(Nexp)
array_Prob_Lin = np.zeros(Nexp)

In [ ]:
# Loop over number of experiments to generate data and subsequent Chi2 values:
for iexp in range( Nexp ) : 

    # Generate points:
    xLin = np.arange(NpointsLin)+1
    yLin = alpha0 + alpha1 * xLin + r.normal(0, sigmay, NpointsLin)
    syLin = sigmay*np.ones_like(xLin)

    def fit_function_Lin(x, alpha0, alpha1):
        return alpha0 + alpha1*x
    
    mfitLin = LeastSquares( xLin, yLin, syLin, fit_function_Lin)
    mfitLin = Minuit(mfitLin, alpha0=1, alpha1=1)  
    mfitLin.migrad();  # perform the actual fit

    Chi2Lin = mfitLin.fval # the chi2 value
    NvarLin = 2                      # Number of variables (alpha0 and alpha1)
    NdofLin = NpointsLin - NvarLin   # Number of degrees of freedom
    ProbLin =  stats.chi2.sf(Chi2Lin, NdofLin) # The chi2 probability given N_DOF degrees of freedom
    
    array_Chi2_Lin[iexp] = Chi2Lin
    array_Prob_Lin[iexp] = ProbLin

In order to inspect the fits, we plot the last one produced:

In [ ]:
figLin, axLin = plt.subplots(figsize=(16, 8))
data_plot = axLin.errorbar(xLin, yLin, syLin, fmt='ro', ecolor='k', elinewidth=1, capsize=2, capthick=1, label="Data")
fit_plot = axLin.plot(xLin, fit_function_Lin(xLin, *mfitLin.values[:]), '-r', label="Fit")
first_legend = axLin.legend(fontsize=18, loc="upper left");

chi2 = mfitLin.fval                           # ChiSquare value
Ndof = len(xLin) - mfitLin.nfit               # Number of points - Number of fit parameters
Prob = stats.chi2.sf(chi2, Ndof)              # ChiSquare probability given Ndof

# Box with "nice" fit information (and correct number of significant decimals):
fit_info = (
    f"$\\chi^2$ / $N_\\mathrm{{dof}}$ = {chi2:.1f} / {Ndof}\n"
    f"Prob($\\chi^2$, $N_\\mathrm{{dof}}$) = {Prob:.3f}\n")
for p, v, e in zip(mfitLin.parameters, mfitLin.values[:], mfitLin.errors[:]) :
    Ndecimals = max(0,-np.int32(np.log10(e)-1-np.log10(2)))                                # Number of significant digits
    fit_info+=(f"{p} = ${v:{10}.{Ndecimals}{"f"}} \\pm {e:{10}.{Ndecimals}{"f"}}$\n")

# Add result of fit on the plot in a separate legend
fit_results, = plt.plot([],[],color='none',label=fit_info)
second_legend = plt.legend(title="Result of fit:", handles=[fit_results], fontsize=18, title_fontsize = 20, alignment = 'left', handlelength=0, handletextpad=0, loc="lower right");
axLin.add_artist(first_legend); # add second legend (with fit info) to the plot

In [ ]:
if (save_plots) : 
    figLin.savefig("Chi2Dist_LinearFit.pdf")

## Generate and fit OSCILLATING data:

In [ ]:
NpointsOsc = 19
mean  = 1.6
amp   = 3.3
omega = 0.7
phase = 0.3
sigmay = 0.5

In [ ]:
# Again we record the resulting Chi2 values and probabilities:
array_Chi2_Osc = np.zeros(Nexp)
array_Prob_Osc = np.zeros(Nexp)

In [ ]:
# Loop over number of experiments to generate data and subsequent Chi2 values:
for iexp in range( Nexp ) : 

    # Generate points:
    xOsc = np.arange(NpointsOsc)+1
    yOsc = mean + amp*np.cos(omega*xOsc + phase) + r.normal(0, sigmay, NpointsOsc)
    syOsc = sigmay*np.ones_like(xOsc)

    # Fit points:
    def fit_function_Osc(x, mean, amp, omega, phase):
        return mean + amp*np.cos(omega*x + phase)
    
    mfitOsc = LeastSquares( xOsc, yOsc, syOsc, fit_function_Osc)
    mfitOsc = Minuit(mfitOsc, mean=mean, amp=amp, omega=omega, phase=phase)  
    mfitOsc.migrad();                # Perform the actual fit

    Chi2Osc = mfitOsc.fval           # Get the chi2 value
    NvarOsc = 4                      # Number of variables (mean, amp, omega, phase)
    NdofOsc = NpointsOsc - NvarOsc   # Number of degrees of freedom    
    ProbOsc =  stats.chi2.sf(Chi2Osc, NdofOsc)   # The chi2 probability given N degrees of freedom, Ndof
    
    array_Chi2_Osc[iexp] = Chi2Osc
    array_Prob_Osc[iexp] = ProbOsc

In [ ]:
figOsc, axOsc = plt.subplots(figsize=(16, 8))
axOsc.errorbar(xOsc, yOsc, syOsc, fmt='ro', ecolor='k', elinewidth=1, capsize=2, capthick=1, label="Data")
xaxis = np.linspace(min(xOsc), max(xOsc), 1000)
axOsc.plot(xaxis, fit_function_Osc(xaxis, *mfitOsc.values[:]), '-r', label="Fit")
first_legend = axOsc.legend(fontsize=18, loc="upper left");

chi2 = mfitOsc.fval                     # ChiSquare value
Ndof = len(xLin[yLin > 0]) - mfitOsc.nfit     # Number of (non-empty) bins
Prob = stats.chi2.sf(chi2, Ndof)     # ChiSquare probability given Ndof

fit_info = (
    f"$\\chi^2$ / $N_\\mathrm{{dof}}$ = {chi2:.1f} / {Ndof}\n"
    f"Prob($\\chi^2$, $N_\\mathrm{{dof}}$) = {Prob:.3f}\n"
)
for p, v, e in zip(mfitOsc.parameters, mfitOsc.values[:], mfitOsc.errors[:]) :
    Ndecimals = max(0,-np.int32(np.log10(e)-1-np.log10(2)))                                # Number of significant digits
    fit_info+=(f"{p} = ${v:{10}.{Ndecimals}{"f"}} \\pm {e:{10}.{Ndecimals}{"f"}}$\n")

# Add result of fit on the plot in a separate legend
fit_results, = plt.plot([],[],color='none',label=fit_info)
second_legend = plt.legend(title="Result of fit:", handles=[fit_results], fontsize=18, title_fontsize = 20, alignment = 'left', handlelength=0, handletextpad=0);
axOsc.add_artist(first_legend);

In [ ]:
if (save_plots) : 
    figOsc.savefig("Chi2Dist_OscillationFit.pdf")

## Generate and fit EXPONENTIAL binned data:

In [ ]:
NpointsExp = 1000   # Put this number of points (exponentially distributed) in each histogram.
NbinsExp = 17       # Decide on the number of bins (for a reason!)

In [ ]:
tau = 3.14           # I'm just picking a random lifetime...

In [ ]:
array_Chi2_Exp = np.zeros(Nexp)
array_Prob_Exp = np.zeros(Nexp)

In [ ]:
# Define an exponential fit function, which includes a normalisation:
def fit_function_Exp(x, tau, N) :
    return N / tau * np.exp(- x / tau)

In [ ]:
# Loop over number of experiments to generate data and subsequent Chi2 values:
for iexp in range( Nexp ) : 
    
    dataExp = r.exponential(tau, NpointsExp)

    yExp, xExp_edges = np.histogram(dataExp, bins=NbinsExp, range=(0, NbinsExp))
    xExp = (xExp_edges[1:] + xExp_edges[:-1])/2
    syExp = np.sqrt(yExp)
    
    mfitExp = LeastSquares( xExp[yExp>0], yExp[yExp>0], syExp[yExp>0], fit_function_Exp)
    mfitExp = Minuit(mfitExp, tau = tau, N=NpointsExp)  
    mfitExp.migrad();  # perform the actual fit

    Chi2Exp = mfitExp.fval
    NvarExp = 2                                   # Number of variables (N and tau)
    NdofExp = len(yExp[yExp > 0]) - NvarExp       # Number of degrees of freedom
    ProbExp =  stats.chi2.sf(Chi2Exp, NdofExp)    # The chi2 probability given Ndof degrees of freedom
    
    array_Chi2_Exp[iexp] = Chi2Exp
    array_Prob_Exp[iexp] = ProbExp

In [ ]:
figExp, axExp = plt.subplots(figsize=(16, 8))
axExp.hist(dataExp, NbinsExp, range=(0, NbinsExp), histtype='step', label="Data")
x_axis = np.linspace(0, NbinsExp, 1000)
axExp.plot(x_axis, fit_function_Exp(x_axis, *mfitExp.values[:]), '-r', label="Fit") 
first_legend = axExp.legend(fontsize=18, loc="upper right");

chi2 = mfitExp.fval                           # ChiSquare value
Ndof = len(xLin[yLin > 0]) - mfitExp.nfit     # Number of (non-empty) bins
Prob = stats.chi2.sf(chi2, Ndof)              # ChiSquare probability given Ndof

fit_info = (
    f"$\\chi^2$ / $N_\\mathrm{{dof}}$ = {chi2:.1f} / {Ndof}\n"
    f"Prob($\\chi^2$, $N_\\mathrm{{dof}}$) = {Prob:.3f}\n"
)

for p, v, e in zip(mfitExp.parameters, mfitExp.values[:], mfitExp.errors[:]) :
    Ndecimals = max(0,-np.int32(np.log10(e)-1-np.log10(2)))                                # Number of significant digits
    fit_info+=(f"{p} = ${v:{10}.{Ndecimals}{"f"}} \\pm {e:{10}.{Ndecimals}{"f"}}$\n")

# Add result of fit on the plot in a separate legend
fit_results, = plt.plot([],[],color='none',label=fit_info)
plt.legend(title="Result of fit:", handles=[fit_results], fontsize=18, title_fontsize = 20, alignment = 'left', handlelength=0, handletextpad=0, loc="right");
axExp.add_artist(first_legend);

In [ ]:
if (save_plots) : 
    figExp.savefig("Chi2Dist_ExponentialFit.pdf")

The above histogram does not show us the uncertainty used in each bin, which the $\chi^2$ needs for its calculation. We have discussed what error to use, and will surely be doing so more in the course, but below is code that gives a plot showing points and errors instead of a "bare" histogram. 

In [ ]:
# Calculating points and errors:
y, bin_edges = np.histogram(dataExp, bins=NbinsExp, range=(0, NbinsExp))
x = 0.5*(bin_edges[:-1] + bin_edges[1:])
sy = np.sqrt(y)                             # Note: Ask yourself (here on in question 4 below) where these errors come from?

# Plotting the result of the above!
fig, ax = plt.subplots(figsize=(16,8))
hist1 = ax.errorbar(x, y, sy, fmt='.', label='Exponential distribution')

# Plot the function we fitted on top? That is a simple exercise for you!

# Considering repeated experiments/fits and resulting $\chi^2$ distributions:

In [ ]:
# If there have been more than one experiment, then make another white canvas:
if (Nexp > 1) :
    
    xaxis = np.linspace(0, 50, 1000)
    yaxis = stats.chi2.pdf(xaxis, 15)   # This is the Chi2 distribution with 15 DOF.
    
    array_Chi2 = [array_Chi2_Lin, array_Chi2_Osc, array_Chi2_Exp]

    fig2, ax2 = plt.subplots(ncols=3, figsize=(16, 6))
    for i in range(3):
        ax2[i].hist(array_Chi2[i], 50, range=(0, 50), histtype='step', density=True)    # Plot normalised!
        ax2[i].plot(xaxis, yaxis)
    
        # Here, we "just" put in quick remarks (note the "code"-like way of defining format. Do you understand it?):
        string  = f"Entries {len(array_Chi2[i]):7d}\n"
        string += f"Mean {array_Chi2[i].mean():10.3f}\n"
        string += f"STD {array_Chi2[i].std(ddof=1):11.3f}"
        ax2[i].text(0.6, 0.95, string, family='monospace', transform=ax2[i].transAxes, fontsize=10, verticalalignment='top')
        
    if (save_plots) : 
        fig2.savefig("Chi2Dist_SeveralCases.pdf")

# Questions:
 1. Why have I chosen the three examples to have 17 points, 19 points and 17 bins?
    Hint: What is the number of degrees of freedom in each of the three cases?

**Example answer 1**:
With this choice of number of points/bins, we made sure that Ndof of all three distributions is the same, namely 15, as the number of fit parameters are 2, 4, and 2, respectively. Therefore, the three $\chi^2$ distributions are (statistically) identical, and follow the ChiSquare distribution for 15 degrees of freedom.

---

 2. In the example of the linear fit, what number of points lies outside +-1 sigma?
    Is that a reasonable number?

**Example answer 2**:
4 out of 15 points (27% of all points). Assuming that uncertainties are Gaussian, 68% of points should like within $\pm 1 \sigma$. Therefore it is a very reasonable number (we have to consider that we have "only" 15 points).

---

 3. In the oscillatory case, try to drop the part were you set the initial parameters
    or simply set them to zero, and see how well the fit goes, when it does not
    have good starting values. How often does it get a good fit result?

**Example answer 3**:
Even in this very simple case with only four parameters, the fit has a hard time finding the minimum, if good input values are not given! We'll return to this point, but whenever a fit fails, you should start by looking at the input parameters. A good option is simply to draw the model on top of the data, to check that it is even remotely close. When it is... release the power of the fit!

---

 4. In the exponential fit, where do the uncertainties come from? And is the fit reasonable?

**Example answer 4**:
The uncertainties come from the assumption, that the number of events in each bin is Poisson distributed (i.e. uncertainties are $\sqrt{N}$), and that the number of entries is high enough, that this can be approximated well with a Gaussian.
This is in fact an approximation, as the bins in the tail do not really have enough statistics for this to hold true. However, these bins are few and in a non-essential place, so the fit should still behave well and give good results. But if the statistics was less and/or the tail longer, one could run into problems.
    

### Advanced questions:
 5. Why does the last of the three Chi2 distributions not fit quite?
    Try to change the number of generated points to 100 instead of 1000,
    and/or change the lifetime to tau=2.1. Does this increase the mismatch
    of the Chi2 distribution. Does that give you a hint why?
    

# Learning points:

This exercise simply gives more ChiSquare fitting illustrations, yet with the same number of degrees of freedom (Ndof) in the fit. From this exercise you should simply get the knowledge of how to run a ChiSquare fit in different situations/cases, calculate Ndof, and be able to extract the p-value.